In [67]:
import re
import time
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=chrome_options)
print(driver.title)

Tania A. Baker - MIT Department of Biology


# Get name and url

In [2]:
def refresh_Pq():
    return pq(driver.page_source)

In [3]:
pi_url_list = []
l = []

In [14]:
doc = refresh_Pq()
pi_list = doc('#post-698 > div.profiles-wrapper > div:nth-child(1)').find('.profile-item').items()
pi_name_url = []
for pi_profile in pi_list:
    name = pi_profile('.first-name').text() + ' ' + pi_profile('.last-name').text()
    url = pi_profile('h3 > a').attr('href')
    pi_name_url.append([name, url])

In [15]:
pi_name_url

[['Tania A. Baker', 'https://biology.mit.edu/profile/tania-a-baker/'],
 ['David Bartel', 'https://biology.mit.edu/profile/david-bartel/'],
 ['Stephen Bell', 'https://biology.mit.edu/profile/stephen-bell/'],
 ['Laurie A. Boyer', 'https://biology.mit.edu/profile/laurie-a-boyer/'],
 ['Christopher Burge', 'https://biology.mit.edu/profile/christopher-burge/'],
 ['Eliezer Calo', 'https://biology.mit.edu/profile/eliezer-calo/'],
 ['Lindsay Case', 'https://biology.mit.edu/profile/lindsay-case/'],
 ['Iain M. Cheeseman', 'https://biology.mit.edu/profile/iain-m-cheeseman/'],
 ['Jianzhu Chen', 'https://biology.mit.edu/profile/jianzhu-chen/'],
 ['Yiyin Erin Chen', 'https://biology.mit.edu/profile/y-erin-chen/'],
 ['Sallie (Penny) W. Chisholm',
  'https://biology.mit.edu/profile/sallie-penny-w-chisholm/'],
 ['Ibrahim Cissé', 'https://biology.mit.edu/profile/ibrahim-cisse/'],
 ['Olivia Corradin', 'https://biology.mit.edu/profile/olivia-corradin/'],
 ['Joseph (Joey) Davis', 'https://biology.mit.edu/pr

In [33]:
from pathlib import Path

RESULT_PATH = Path.cwd().parents[1] / 'result/mit'
RESULT_PATH.exists()

True

## crawl mit pi personal information

In [120]:
def extract_one_pate_pi_info(pi_link):
    try:
        persoanl_page = pq(pi_link, encoding='utf-8', timeout=30)
        pass
    except:
        print(pi_link)
        return np.nan, np.nan, np.nan,
    else:
        degrees = list(
            persoanl_page('#main > div.profile-full-content > div > div > div.page-content  h2:nth-child(1)').eq(
                0).siblings('ul').eq(0).children().items())
        # ed_title.sli
        # degrees=list(persoanl_page('#main > div.profile-full-content > div > div > div.page-content  ul > li').items())
        # degrees=list(persoanl_page('#main > div.profile-full-content > div > div > div.page-content > ul:nth-child(2) > li:nth-child(1)').items())
        if (degrees_num := len(degrees)) <= 1:
            print('lesson 1', pi_link)
        md, bd, bd_y, md_y = np.nan, np.nan, np.nan, np.nan,
        year = re.compile(u'\d{4}')
        try:
            phd = degrees[0].text().split(', ')[-1]
            phd_y = year.search(degrees[0].text()).group()
        except:
            print(pi_link, '!!!!!!!!!!!!!!!!!!!!')
            return np.nan, np.nan, np.nan,

        else:
            if degrees_num == 3:
                md = degrees[1].text().split(', ')[-1]
                md_y = year.search(degrees[1].text()).group()

                bd = degrees[2].text().split(', ')[-1]
                bd_y = year.search(degrees[2].text()).group()

            elif degrees_num == 2:
                bd = degrees[-1].text().split(', ')[-1]
                bd_y = year.search(degrees[-1].text()).group()

            return bd, bd_y, md, md_y, phd, phd_y


bd, bd_y, md, md_y, phd, phd_y = extract_one_pate_pi_info('https://biology.mit.edu/profile/gene-wei-li/')
print(bd, bd_y, md, md_y, phd, phd_y)

National Tsinghua University 2004 nan nan Harvard University 2010


In [122]:
begin = time.time()
executor = ThreadPoolExecutor(max_workers=40)
pi_names, pi_urls = zip(*pi_name_url)
records = []
for infos, author_name in tqdm(zip(executor.map(extract_one_pate_pi_info, pi_urls), pi_names)):
    records.append([author_name, *infos ])
times = time.time() - begin
print(times, 's')

3it [00:07,  1.96s/it]

lesson 1 https://biology.mit.edu/profile/sallie-penny-w-chisholm/


16it [00:10,  2.11it/s]

lesson 1 https://biology.mit.edu/profile/chris-a-kaiser/


24it [00:13,  2.32it/s]

lesson 1 https://biology.mit.edu/profile/jonathan-weissman/
https://biology.mit.edu/profile/jonathan-weissman/ !!!!!!!!!!!!!!!!!!!!


27it [00:15,  2.21it/s]

lesson 1 https://biology.mit.edu/profile/rudolf-jaenisch/


73it [00:19,  3.80it/s]

19.28064250946045 s


In [124]:
df = pd.DataFrame.from_records(records,columns=['name','bd', 'bd_y', 'md', 'md_y', 'phd', 'phd_y' ])
df

,name,bd,bd_y,md,md_y,phd,phd_y
0,Tania A. Baker,University of Wisconsin-Madison,1983,NaN,NaN,Stanford University,1988
1,David Bartel,Goshen College,1982,NaN,NaN,Harvard University,1993
2,Stephen Bell,Northwestern University,1985,NaN,NaN,Berkeley,1990
3,Laurie A. Boyer,Framingham State University,1990,NaN,NaN,University of Massachusetts Medical School,2001
4,Christopher Burge,Stanford University,1990,NaN,NaN,Stanford University,1997
...,...,...,...,...,...,...,...
68,Harikesh S. Wong,McMaster University,2010,NaN,NaN,University of Toronto,2016
69,Michael B. Yaffe,Cornell University,1981,NaN,NaN,Case Western Reserve University,1987
70,Yukiko Yamashita,Kyoto University,1994,NaN,NaN,Kyoto University,1999
71,Omer H. Yilmaz,University of Michigan,1999,NaN,NaN,University of Michigan Medical School,2008


### save the pi info

In [127]:
df.to_csv(RESULT_PATH/'mit_faculty_degrees_info.csv',index=False, encoding='utf8')

## crawl its google scholar page